In [46]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations

In [47]:
#from google.colab import files
#uploaded = files.upload()
data = pd.read_csv('movie_bd_v5.csv')

In [48]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [49]:
def is_profit(profit): #функция для того чтобы проставить признак "выгодный фильм или нет"
  if profit >0:
    return 1
  else: return 0

answers = {} # создадим словарь для ответов

#Создаём новые колонки для удобства дальнейших подсчётов
data['profit'] = data['revenue'] - data['budget'] #создадим новую колонку с Профитом
data['is_profit'] = data.profit.apply(is_profit) #создадим новую колонку с признаком выгодности фильма
data['director_list'] = data['director'].apply(lambda x: x.split('|')) #вынесем список режисёров отдельно
data['cast_list'] = data['cast'].apply(lambda x: x.split('|'))  #тоже самое для актёров

#тут апдейт существующих колонок. Минусом - повторный запуск этого же куска кода приводит к ошибкам. Вероятно лучше поменять на создание новых колонок.
data['production_companies'] = data['production_companies'].apply(lambda x:x.split('|')) #и компаний
data['release_date'] = data['release_date'].apply(lambda x: x.split('/')) #И создаём колонку для месяца. Сперва разбиваем дату релиза на массив
data['month'] = data['release_date'].apply(lambda x: x[0]) #потом первым значением из массива апдейтим колонку с месяцем


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [50]:
answers['1'] = (data['original_title'].loc[data['budget'] == data['budget'].max()])

# 2. Какой из фильмов самый длительный (в минутах)?

In [51]:
answers['2'] = (data['original_title'].loc[data['runtime'] == data['runtime'].max()])

# 3. Какой из фильмов самый короткий (в минутах)?





In [52]:
answers['3'] = (data['original_title'].loc[data['runtime'] == data['runtime'].min()])

# 4. Какова средняя длительность фильмов?


In [53]:
answers['4'] = data['runtime'].mean()

# 5. Каково медианное значение длительности фильмов? 

In [54]:
answers['5'] = data['runtime'].median()

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [55]:
answers['6'] = (data['original_title'].loc[data['profit'] == data['profit'].max()])

# 7. Какой фильм самый убыточный? 

In [56]:
answers['7'] = (data['original_title'].loc[data['profit'] == data['profit'].min()])

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [57]:
answers['8'] = (data['imdb_id'].loc[data['revenue'] > data['budget']].count())

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [58]:
data1 = data.query('release_year == 2008')
answers['9'] = data1['original_title'].loc[data1['revenue'] == data1['revenue'].max()]

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [59]:
data1 = data.query('2012 <= release_year <= 2014')
answers['10'] = data1['original_title'].loc[data1['profit'] == data1['profit'].min()]

# 11. Какого жанра фильмов больше всего?

In [60]:
#Создаём большой массив куда добавляем все значения найденных жанров. Плохое решение для миллионов строк.
genres_list = [] 
for rows in data['genres']:
  genres_list= genres_list+rows.split('|')

#Находим самое частовстречаемое значение
answers['11'] = Counter(genres_list).most_common(1)

ВАРИАНТ 2

In [61]:
#Тoже что выше но превратить список в датафрейм
genres_list = []
for rows in data['genres']:
  genres_list= genres_list+rows.split('|')

gendres_df = pd.DataFrame(genres_list,columns=['genre'])
gendres_df['genre'].mode()

0    Drama
dtype: object

ВАРИАНТ 3

In [62]:
data.genres.apply(lambda x:x.split(sep = '|')).explode().value_counts().head(1) #Более правильный вариант. Основан на решении №12

Drama    782
Name: genres, dtype: int64

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [63]:
answers['12'] = data.genres.loc[data['is_profit']==1].apply(lambda x:x.split(sep = '|')).explode().value_counts().head(1)

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [64]:
answers['13'] = data.explode('director_list').groupby(['director_list'])['revenue'].sum().sort_values(ascending=False).head(1)

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [65]:
answers['14'] = data.explode('director_list').loc[data['genres'].str.contains('Action', regex = False)].groupby(['director_list'])['imdb_id'].count().sort_values(ascending=False).head(1)

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [66]:
data1 = data.query('release_year == 2012')
answers['15'] = data1.explode('cast_list').groupby('cast_list')['revenue'].sum().sort_values(ascending=False).head(1)

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [67]:
answers['16'] = data.cast_list.loc[data['budget'] >= data['budget'].mean()].explode('cast_list').value_counts().head(1)

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [68]:
answers['17'] = data.genres.apply(lambda x:x.split(sep = '|')).explode().loc[data['cast_list'].str.contains('Nicolas Cage', regex = False)].value_counts().head(1)

# 18. Самый убыточный фильм от Paramount Pictures

In [69]:
answers['18'] = data.loc[data['production_companies'].str.contains('Paramount Pictures', regex = False)].sort_values(by = ['profit']).head(1).original_title

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [70]:
answers['19'] = data.groupby(['release_year'])['revenue'].sum().sort_values(ascending=False).head(1)

# 20. Какой самый прибыльный год для студии Warner Bros?

In [71]:
data1 = data.explode('production_companies').copy()
answers['20'] = data1.loc[data1['production_companies'].str.contains('Warner Bros.', regex = False)].groupby(['release_year'])['profit'].sum().sort_values(ascending = False).head(1)

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [72]:
answers['21'] = data.month.value_counts().head(1)

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [73]:
answers['22'] = data.imdb_id.loc[((data['month'] == '6') | (data['month'] == '7') | (data['month'] == '8'))].count()

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [74]:
answers['23'] = data.explode('director_list').loc[((data['month'] == '12') | (data['month'] == '1') | (data['month'] == '2'))].groupby('director_list')['imdb_id'].count().sort_values(ascending = False).head(1)

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [75]:
data1 = data.copy()
data1['title_lenth'] = data1['original_title'].apply(lambda x: len(x))
answers['24'] = data1.loc[data1['title_lenth'] == data1['title_lenth'].max()].production_companies.explode().head()

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [76]:
data1['desc_lenth'] = data1['overview'].apply(lambda x: len(x))
answers['25']= data1.explode('production_companies').groupby(['production_companies'])['desc_lenth'].mean().sort_values(ascending = False).head(1)

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [77]:
q99 = data['vote_average'].quantile(0.99)
answers['26'] = data.loc[data['vote_average'] >= q99].sort_values(by = 'vote_average', ascending= False).original_title.head(7).tolist()

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [78]:
data1 = data.explode('cast_list').copy() #Сделали копию списка со всеми актёрами в колонке
joined_frame = data1.merge(data.explode('cast_list'), on='imdb_id', how='left') #Получили все возможные комбинации актёров в фильме в том числе сам с собой
joined_frame.query('cast_list_x != cast_list_y') #запросом удаляем явные дубли когда актёр играл сам ссобой
answers['27'] = joined_frame.query('cast_list_x != cast_list_y').groupby(['cast_list_x', 'cast_list_y'])['imdb_id'].count().sort_values(ascending=False).head(3) 
#Минус такого решения в том что пары дублируются. Надо или хитрым способом удалять дубли или itertools.combinations

Вариант2

In [79]:
data['pairs'] = data['cast_list'].apply(lambda s: list(combinations(s, 2)))
Counter(data['pairs'].explode('pairs')).most_common(3)

[(('Daniel Radcliffe', 'Rupert Grint'), 8),
 (('Daniel Radcliffe', 'Emma Watson'), 8),
 (('Rupert Grint', 'Emma Watson'), 7)]

# Submission

In [83]:
# в конце можно посмотреть свои ответы к каждому вопросу
print(answers)

{'1': 723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object, '2': 1157    Gods and Generals
Name: original_title, dtype: object, '3': 768    Winnie the Pooh
Name: original_title, dtype: object, '4': 109.6585494970884, '5': 107.0, '6': 239    Avatar
Name: original_title, dtype: object, '7': 1245    The Lone Ranger
Name: original_title, dtype: object, '8': 1478, '9': 599    The Dark Knight
Name: original_title, dtype: object, '10': 1245    The Lone Ranger
Name: original_title, dtype: object, '11': [('Drama', 782)], '12': Drama    560
Name: genres, dtype: int64, '13': director_list
Peter Jackson    6490593685
Name: revenue, dtype: int64, '14': director_list
Robert Rodriguez    9
Name: imdb_id, dtype: int64, '15': cast_list
Chris Hemsworth    2027450773
Name: revenue, dtype: int64, '16': Matt Damon    18
Name: cast_list, dtype: int64, '17': Action    17
Name: genres, dtype: int64, '18': 925    K-19: The Widowmaker
Name: original_title, dtype: object, '19': 

In [82]:
# и убедиться что ни чего не пропустил)
len(answers)

27